# 2. 경사 하강법을 이용한 얕은 신경망 학습 

In [1]:
import tensorflow as tf
import numpy as np

## 1) 하이퍼 파라미터 설정

In [2]:
EPOCHS = 1000

## 2) 네트워크 구조 정의 - 얕은 신경망 
이번에는 numpy가 아닌 tensorflow를 이용하여서 얕은 신경망 구조를 정의 할 것이다. 

**input layers: 2, hidden layers: 128(sigmoid activation), output layers: 10(Softmax activation,즉, 10개의 class로 classification될 것이다.)**

In [3]:
class Mymodel(tf.keras.Model) :  #tensorflow2.0에서 네트워크 구조를 정의하기 위해서는 keras에 있는 모델을 상속해서 class를 구현한다.  
    def __init__(self):#그리고 두 가지 function을 정의해야 한다. 그중 첫 번째가 initialize function이다. 
        super(Mymodel, self).__init__()
        self.denselayer1 = tf.keras.layers.Dense(128, input_dim = 2, activation = 'sigmoid') #keras에서는 fully connected layer를 'Dense'라고 표현한다. 
        self.denselayer2 = tf.keras.layers.Dense(10, input_dim = 128, activation = 'softmax')
            
    def call(self, x, training = None, mask = None) : #그 다음은 call function으로, 모델이 실제 call이 될때 입력에서 출력으로 어떻게 연결이 될 것인지 정의한다.
        x = self.denselayer1(x)
        return self.denselayer2(x)
        

## 3) 학습루프 정의
tensorflow의 outograph를 이용해서 쉽게 구현가능

In [4]:
@tf.function 
#tf.fucnction이라는 데코레이터를 이용하면 train_step()안에 있는 모든 tensor연산들이 최적화된다. 

def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
     #train_step에서 가장 중요한 것은 Gradient를 계산해서 optimizer에 넣어서 학습을 진행하는 것이다. 
    with tf.GradientTape() as tape : #GradientTape : 이 내에서 계산되는 모든 것의 gradient가 계산되어 tape안에 들어있게 된다. 
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)#loss를 model.trainable_variables로 미분해서 구해준 gradient가 gradients로 들어간다. 
    
    #이렇게 gradients를 구한 후 optimizer에 반영을 한다. 
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_metric(label, predictions) #metric을 이용해서 label과 prediction을 비교해서 평가

## 4) 데이터셋 생성 및 전처리 - numpy이용

In [5]:
#데이터셋으르 직접 만들어서 사용해보자. 
np.random.seed(0)  #seed지정해두면 프로그램 실행 할 때마다 동일한 랜덤값이 생성되어 결과값 확인을 반복할 수 있다. 

컴퓨터 프로그램에서 발생하는 무작위 수는 사실 엄격한 의미의 무작위 수가 아니다. 어떤 특정한 시작 숫자를 정해 주면 컴퓨터가 정해진 알고리즘에 의해 마치 난수처럼 보이는 수열을 생성한다. 이런 시작 숫자를 시드(seed)라고 한다. 일단 생성된 난수는 다음번 난수 생성을 위한 시드값이 된다. 따라서 시드값은 한 번만 정해주면 된다. 시드는 보통 현재 시각등을 이용하여 자동으로 정해지지만 사람이 수동으로 설정할 수도 있다. 특정한 시드값이 사용되면 그 다음에 만들어지는 난수들은 모두 예측할 수 있다. 이 책에서는 코드의 결과를 재현하기 위해 항상 시드를 설정한다.

파이썬에서 시드를 설정하는 명령은 seed이다. 인수로는 0과 같거나 큰 정수를 넣어준다.

In [6]:
points = list() #입력값 input layers: 2
labels = list() #출력값 output layers: 10(Softmax activation,즉, 10개의 class로 classification될 것이다.
#입력값으로 쓸 points들은 가우시안 분포로 10개의 labels에 포함되도록 퍼트려 줄 것이다. 
center_points = np.random.uniform(-8.0, 8.0, (10, 2))

for label, center_point in enumerate(center_points): 
    for _ in range(100):
        points.append(center_point + np.random.randn(*center_point.shape))
        labels.append(label)
        
points = np.stack(points, axis= 0).astype(np.float32)
lables = np.stack(labels, axis = 0)

train_ds = tf.data.Dataset.from_tensor_slices((points, labels)).shuffle(10000).batch(32)

## 5) 모델 생성
모델생성은 앞의 <2) 네트워크 구조 정의 - 얕은 신경망>에서' class Mymodel(tf.keras.Model) : '이라고 해 줬기 때문에 

In [7]:
model = Mymodel()

## 6) 손실 함수(Cross Entropy사용) 및 최적화 알고리즘(Adam Optimizer 사용) 설정

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() #Cross Entropy는 keras에서 정의한다. 
optimizer = tf.keras.optimizers.Adam()

## 7) 평가 지표(Accuracy 사용) 설정

In [9]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

## 8) 학습 시키기

In [10]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        #def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):라서 
    
        template = 'EPOCH{}, Loss: {}, Accuracy: {}'
    print(template.format(epoch+1, train_loss.result(), train_accuracy.result()*100))
        
         

EPOCH1, Loss: 2.219269037246704, Accuracy: 11.5
EPOCH2, Loss: 2.0229384899139404, Accuracy: 10.100000381469727
EPOCH3, Loss: 1.88105046749115, Accuracy: 9.600000381469727
EPOCH4, Loss: 1.7677063941955566, Accuracy: 9.425000190734863
EPOCH5, Loss: 1.6739448308944702, Accuracy: 9.359999656677246
EPOCH6, Loss: 1.5955314636230469, Accuracy: 9.5
EPOCH7, Loss: 1.5269004106521606, Accuracy: 9.171428680419922
EPOCH8, Loss: 1.467152714729309, Accuracy: 9.112499237060547
EPOCH9, Loss: 1.4133044481277466, Accuracy: 9.188888549804688
EPOCH10, Loss: 1.3646514415740967, Accuracy: 9.319999694824219
EPOCH11, Loss: 1.3208965063095093, Accuracy: 9.409090995788574
EPOCH12, Loss: 1.2805362939834595, Accuracy: 9.458333015441895
EPOCH13, Loss: 1.2437875270843506, Accuracy: 9.284615516662598
EPOCH14, Loss: 1.209771990776062, Accuracy: 9.307143211364746
EPOCH15, Loss: 1.1783233880996704, Accuracy: 9.359999656677246
EPOCH16, Loss: 1.1486411094665527, Accuracy: 9.40625
EPOCH17, Loss: 1.1210508346557617, Accurac

EPOCH133, Loss: 0.4517691433429718, Accuracy: 9.954886436462402
EPOCH134, Loss: 0.4504733085632324, Accuracy: 9.96194076538086
EPOCH135, Loss: 0.4492036700248718, Accuracy: 9.95111083984375
EPOCH136, Loss: 0.44796058535575867, Accuracy: 9.961764335632324
EPOCH137, Loss: 0.4467039704322815, Accuracy: 9.951825141906738
EPOCH138, Loss: 0.44545650482177734, Accuracy: 9.944927215576172
EPOCH139, Loss: 0.44423627853393555, Accuracy: 9.94460391998291
EPOCH140, Loss: 0.4430038034915924, Accuracy: 9.943572044372559
EPOCH141, Loss: 0.44180434942245483, Accuracy: 9.946099281311035
EPOCH142, Loss: 0.4406047463417053, Accuracy: 9.947887420654297
EPOCH143, Loss: 0.43947720527648926, Accuracy: 9.940559387207031
EPOCH144, Loss: 0.43833351135253906, Accuracy: 9.934721946716309
EPOCH145, Loss: 0.43718379735946655, Accuracy: 9.948965072631836
EPOCH146, Loss: 0.4360768496990204, Accuracy: 9.949999809265137
EPOCH147, Loss: 0.43497219681739807, Accuracy: 9.944897651672363
EPOCH148, Loss: 0.43392738699913025

EPOCH261, Loss: 0.3615480661392212, Accuracy: 10.036398887634277
EPOCH262, Loss: 0.36117586493492126, Accuracy: 10.029389381408691
EPOCH263, Loss: 0.360798716545105, Accuracy: 10.029658317565918
EPOCH264, Loss: 0.3604156970977783, Accuracy: 10.030681610107422
EPOCH265, Loss: 0.36003682017326355, Accuracy: 10.030566215515137
EPOCH266, Loss: 0.3596527576446533, Accuracy: 10.025188446044922
EPOCH267, Loss: 0.35929107666015625, Accuracy: 10.023595809936523
EPOCH268, Loss: 0.35890889167785645, Accuracy: 10.022015571594238
EPOCH269, Loss: 0.3585284948348999, Accuracy: 10.023048400878906
EPOCH270, Loss: 0.35820290446281433, Accuracy: 10.02148151397705
EPOCH271, Loss: 0.35784244537353516, Accuracy: 10.023984909057617
EPOCH272, Loss: 0.3574959933757782, Accuracy: 10.026838302612305
EPOCH273, Loss: 0.3571290373802185, Accuracy: 10.031867980957031
EPOCH274, Loss: 0.3567676544189453, Accuracy: 10.022262573242188
EPOCH275, Loss: 0.3564448058605194, Accuracy: 10.023272514343262
EPOCH276, Loss: 0.356

EPOCH388, Loss: 0.328249990940094, Accuracy: 9.995360374450684
EPOCH389, Loss: 0.32808005809783936, Accuracy: 9.994087219238281
EPOCH390, Loss: 0.3278936445713043, Accuracy: 9.99128246307373
EPOCH391, Loss: 0.3277127742767334, Accuracy: 9.990537643432617
EPOCH392, Loss: 0.32755595445632935, Accuracy: 9.990560531616211
EPOCH393, Loss: 0.32738012075424194, Accuracy: 9.989822387695312
EPOCH394, Loss: 0.32719236612319946, Accuracy: 9.992639541625977
EPOCH395, Loss: 0.3270222246646881, Accuracy: 9.994683265686035
EPOCH396, Loss: 0.32684919238090515, Accuracy: 9.995959281921387
EPOCH397, Loss: 0.32666710019111633, Accuracy: 9.994709968566895
EPOCH398, Loss: 0.3264838755130768, Accuracy: 9.995980262756348
EPOCH399, Loss: 0.32634106278419495, Accuracy: 9.995237350463867
EPOCH400, Loss: 0.3261721432209015, Accuracy: 9.991000175476074
EPOCH401, Loss: 0.32601654529571533, Accuracy: 9.989027976989746
EPOCH402, Loss: 0.32584017515182495, Accuracy: 9.994278907775879
EPOCH403, Loss: 0.325660079717636

EPOCH518, Loss: 0.3103597164154053, Accuracy: 9.978185653686523
EPOCH519, Loss: 0.31024423241615295, Accuracy: 9.979961395263672
EPOCH520, Loss: 0.31014493107795715, Accuracy: 9.976922988891602
EPOCH521, Loss: 0.3100601136684418, Accuracy: 9.975239753723145
EPOCH522, Loss: 0.309947669506073, Accuracy: 9.97452163696289
EPOCH523, Loss: 0.3098524212837219, Accuracy: 9.973804473876953
EPOCH524, Loss: 0.3097566068172455, Accuracy: 9.972328186035156
EPOCH525, Loss: 0.3096558749675751, Accuracy: 9.973333358764648
EPOCH526, Loss: 0.30955770611763, Accuracy: 9.973003387451172
EPOCH527, Loss: 0.30944547057151794, Accuracy: 9.971916198730469
EPOCH528, Loss: 0.30934938788414, Accuracy: 9.974052429199219
EPOCH529, Loss: 0.3092425465583801, Accuracy: 9.973345756530762
EPOCH530, Loss: 0.3091299533843994, Accuracy: 9.97188663482666
EPOCH531, Loss: 0.30902591347694397, Accuracy: 9.969868659973145
EPOCH532, Loss: 0.30893146991729736, Accuracy: 9.971616744995117
EPOCH533, Loss: 0.30882054567337036, Accur

EPOCH646, Loss: 0.2991499900817871, Accuracy: 9.96842098236084
EPOCH647, Loss: 0.2990801930427551, Accuracy: 9.969243049621582
EPOCH648, Loss: 0.2990061640739441, Accuracy: 9.968364715576172
EPOCH649, Loss: 0.298935204744339, Accuracy: 9.966256141662598
EPOCH650, Loss: 0.298854798078537, Accuracy: 9.966769218444824
EPOCH651, Loss: 0.2987886667251587, Accuracy: 9.968356132507324
EPOCH652, Loss: 0.2987136244773865, Accuracy: 9.96901798248291
EPOCH653, Loss: 0.29863405227661133, Accuracy: 9.969372749328613
EPOCH654, Loss: 0.29855287075042725, Accuracy: 9.970030784606934
EPOCH655, Loss: 0.2984779477119446, Accuracy: 9.970229148864746
EPOCH656, Loss: 0.2984083294868469, Accuracy: 9.969511985778809
EPOCH657, Loss: 0.2983333170413971, Accuracy: 9.968340873718262
EPOCH658, Loss: 0.2982698678970337, Accuracy: 9.967628479003906
EPOCH659, Loss: 0.298207551240921, Accuracy: 9.965401649475098
EPOCH660, Loss: 0.2981516420841217, Accuracy: 9.964393615722656
EPOCH661, Loss: 0.2980744540691376, Accurac

EPOCH775, Loss: 0.29114243388175964, Accuracy: 9.957161903381348
EPOCH776, Loss: 0.29109182953834534, Accuracy: 9.958633422851562
EPOCH777, Loss: 0.29104095697402954, Accuracy: 9.957786560058594
EPOCH778, Loss: 0.29098305106163025, Accuracy: 9.957069396972656
EPOCH779, Loss: 0.29093262553215027, Accuracy: 9.956868171691895
EPOCH780, Loss: 0.290897011756897, Accuracy: 9.957307815551758
EPOCH781, Loss: 0.2908405363559723, Accuracy: 9.956337928771973
EPOCH782, Loss: 0.29078376293182373, Accuracy: 9.958695411682129
EPOCH783, Loss: 0.29073694348335266, Accuracy: 9.957087516784668
EPOCH784, Loss: 0.29068249464035034, Accuracy: 9.9573974609375
EPOCH785, Loss: 0.29062673449516296, Accuracy: 9.957451820373535
EPOCH786, Loss: 0.2905726432800293, Accuracy: 9.956997871398926
EPOCH787, Loss: 0.29051804542541504, Accuracy: 9.95679759979248
EPOCH788, Loss: 0.2904696464538574, Accuracy: 9.957740783691406
EPOCH789, Loss: 0.29041483998298645, Accuracy: 9.957413673400879
EPOCH790, Loss: 0.290358603000640

EPOCH903, Loss: 0.28516635298728943, Accuracy: 9.964008331298828
EPOCH904, Loss: 0.28512969613075256, Accuracy: 9.962942123413086
EPOCH905, Loss: 0.2850882411003113, Accuracy: 9.96364688873291
EPOCH906, Loss: 0.2850452959537506, Accuracy: 9.96335506439209
EPOCH907, Loss: 0.28500238060951233, Accuracy: 9.96229362487793
EPOCH908, Loss: 0.28495508432388306, Accuracy: 9.963105201721191
EPOCH909, Loss: 0.28491419553756714, Accuracy: 9.962045669555664
EPOCH910, Loss: 0.28487083315849304, Accuracy: 9.962857246398926
EPOCH911, Loss: 0.2848328649997711, Accuracy: 9.963227272033691
EPOCH912, Loss: 0.2847900092601776, Accuracy: 9.96260929107666
EPOCH913, Loss: 0.28475242853164673, Accuracy: 9.961773872375488
EPOCH914, Loss: 0.2847258448600769, Accuracy: 9.964004516601562
EPOCH915, Loss: 0.28468337655067444, Accuracy: 9.963605880737305
EPOCH916, Loss: 0.28463825583457947, Accuracy: 9.963318824768066
EPOCH917, Loss: 0.2845923602581024, Accuracy: 9.962267875671387
EPOCH918, Loss: 0.2845514118671417,

## 9) numpy를 이용해서 데이터셋 및 학습 파라미터 저장

In [12]:
np.savez_compressed('ch2_dataset.npz', inputs=points, labels=labels)
#savez_compressed : 용량을 압축해서 저장한다. 

W_hiddenlayer, b_hiddenlayer = model.denselayer1.get_weights()
W_outputlayer, b_outputlayer = model.denselayer2.get_weights()
W_hiddenlayer = np.transpose(W_hiddenlayer)
W_outputlayer = np.transpose(W_outputlayer)
np.savez_compressed('ch2_parameters.npz',
                    W_hiddenlayer=W_hiddenlayer,
                    b_hiddenlayer=b_hiddenlayer,
                    W_outputlayer=W_outputlayer,
                    b_outputlayer=b_outputlayer)